In [1]:
import os
import numpy as np
import pandas as pd

from config import (
    FLOOR_AREA, BUILDING_VOLUME, VENT_FLOW, HRV_EFF, INFILTRATION_ACH,
    INTERNAL_GAIN_W, ALPHA, EPSILON, SHGC,
    WALL_R, ROOF_R, WINDOW_R,
    AREA_ROOF, AREA_WALL_N, AREA_WALL_S, AREA_WALL_E, AREA_WALL_W, AREA_WINDOW
)
from src.building import Plane, AirSide, InternalGains
from src.physics import run_hourly
from src.weather import load_epw_weather
from src.results import (
    plot_fig2_monthly,
    plot_fig2_2_breakdown, 
    plot_fig3_pies, 
    plot_fig4_stacked,
    plot_fig4_2_gains, 
    plot_fig5_winter, 
    plot_fig6_shoulder
    )

from src.sensitivity import (
    run_sensitivity, 
    calc_sensitivity_table,
    plot_fig7_scatter, 
    plot_fig8_ranking
    )

os.makedirs('outputs', exist_ok=True)

In [2]:
# Load weather data
weather: pd.DataFrame = load_epw_weather(
    'weather_files/GBR_SCT_Glasgow.AP.031400_TMYx/GBR_SCT_Glasgow.AP.031400_TMYx.epw',
    'weather_files/solarposition_data_Glasgow.csv'
)
weather['timestamp'] = pd.to_datetime({
    'year': 2021,
    'month': weather['timestamp'].dt.month,
    'day': weather['timestamp'].dt.day,
    'hour': weather['timestamp'].dt.hour
})
weather = weather.sort_values('timestamp').reset_index(drop=True)

In [3]:
def setpoint_schedule(timestamps, constant: bool = True) -> np.ndarray:
    hours: np.ndarray = pd.Series(timestamps).dt.hour.values
    if constant:
        return np.full(len(hours), 21.0)
    return np.where((hours >= 7) & (hours < 23), 21.0, 18.0)


def vent_schedule(timestamps, constant: bool = True) -> np.ndarray:
    hours: np.ndarray = pd.Series(timestamps).dt.hour.values
    if constant:
        return np.full(len(hours), 0.5)
    return np.where((hours >= 7) & (hours < 23), 0.7, 0.3)

In [4]:
# Building envelope
roof = Plane('Roof', 'opaque', AREA_ROOF, 0, 0, ROOF_R, ALPHA, EPSILON)
wall_N = Plane('Wall-N', 'opaque', AREA_WALL_N, 90, 0, WALL_R, ALPHA, EPSILON)
wall_S = Plane('Wall-S', 'opaque', AREA_WALL_S, 90, 180, WALL_R, ALPHA, EPSILON)
wall_E = Plane('Wall-E', 'opaque', AREA_WALL_E, 90, 90, WALL_R, ALPHA, EPSILON)
wall_W = Plane('Wall-W', 'opaque', AREA_WALL_W, 90, 270, WALL_R, ALPHA, EPSILON)
window_S = Plane('Win-S', 'window', AREA_WINDOW, 90, 180, WINDOW_R, g=SHGC)

planes = [roof, wall_N, wall_S, wall_E, wall_W, window_S]
air = AirSide(BUILDING_VOLUME, VENT_FLOW, HRV_EFF, INFILTRATION_ACH)
gains = InternalGains(INTERNAL_GAIN_W / 1000)

# Task 2

In [5]:
results = {}

# S1: constant setpoint, constant ventilation
T_set_S1: np.ndarray = setpoint_schedule(weather['timestamp'], constant=True)
vent_S1: np.ndarray = vent_schedule(weather['timestamp'], constant=True)
results['S1'] = run_hourly(weather, planes, air, T_set_S1, vent_ACH=vent_S1, gains=gains)
kwh_S1 = results['S1']['Q_heat_W'].sum() / 1000
print(f"  S1: {kwh_S1:.0f} kWh ({kwh_S1/FLOOR_AREA:.1f} kWh/m²)")

# S2: variable setpoint, constant ventilation
T_set_S2: np.ndarray = setpoint_schedule(weather['timestamp'], constant=False)
vent_S2: np.ndarray = vent_schedule(weather['timestamp'], constant=True)
results['S2'] = run_hourly(weather, planes, air, T_set_S2, vent_ACH=vent_S2, gains=gains)
kwh_S2 = results['S2']['Q_heat_W'].sum() / 1000
print(f"  S2: {kwh_S2:.0f} kWh ({kwh_S2/FLOOR_AREA:.1f} kWh/m²)")

# S3: constant setpoint, variable ventilation
T_set_S3: np.ndarray = setpoint_schedule(weather['timestamp'], constant=True)
vent_S3: np.ndarray = vent_schedule(weather['timestamp'], constant=False)
results['S3'] = run_hourly(weather, planes, air, T_set_S3, vent_ACH=vent_S3, gains=gains)
kwh_S3 = results['S3']['Q_heat_W'].sum() / 1000
print(f"  S3: {kwh_S3:.0f} kWh ({kwh_S3/FLOOR_AREA:.1f} kWh/m²)")

# S4: variable setpoint, variable ventilation
T_set_S4: np.ndarray = setpoint_schedule(weather['timestamp'], constant=False)
vent_S4: np.ndarray = vent_schedule(weather['timestamp'], constant=False)
results['S4'] = run_hourly(weather, planes, air, T_set_S4, vent_ACH=vent_S4, gains=gains)
kwh_S4 = results['S4']['Q_heat_W'].sum() / 1000
print(f"  S4: {kwh_S4:.0f} kWh ({kwh_S4/FLOOR_AREA:.1f} kWh/m²)")

  S1: 8275 kWh (172.4 kWh/m²)
  S2: 7217 kWh (150.4 kWh/m²)
  S3: 8317 kWh (173.3 kWh/m²)
  S4: 7330 kWh (152.7 kWh/m²)


In [6]:
timestamps = results['S1']['timestamp']
winter_mask = (timestamps >= '2021-01-08') & (timestamps < '2021-01-15')
shoulder_mask = (timestamps >= '2021-10-07') & (timestamps < '2021-10-14')

print("\nHeat Flow Breakdown (S1)")
print("-" * 50)

s1 = results["S1"]
losses = {
    "Walls": s1["Q_walls_W"].sum() / 1000,
    "Roof": s1["Q_roof_W"].sum() / 1000,
    "Windows": s1["Q_win_W"].sum() / 1000,
    "Infiltration": s1["Q_inf_W"].sum() / 1000,
    "Ventilation": s1["Q_vent_W"].sum() / 1000,
}
gains = {
    "Solar": s1["Q_solar_W"].sum() / 1000,
    "Internal": s1["Q_int_W"].sum() / 1000,
    "Heating": s1["Q_heat_W"].sum() / 1000,
}
total_loss = sum(losses.values())
total_gain = sum(gains.values())

print("Losses:")
for k, v in losses.items():
    print(f"  {k}: {v:.0f} kWh ({v/total_loss*100:.1f}%)")
print(f"  Total: {total_loss:.0f} kWh")

print("Gains:")
for k, v in gains.items():
    print(f"  {k}: {v:.0f} kWh ({v/total_gain*100:.1f}%)")
print(f"  Total: {total_gain:.0f} kWh")

plot_fig2_monthly(results, weather, 'outputs/fig2_monthly.png')
plot_fig2_2_breakdown(results, weather, 'outputs/fig2_2_breakdown.png')
plot_fig3_pies(results, 'outputs/fig3_breakdown.png')
plot_fig4_stacked(results, 'outputs/fig4_comparison.png')
plot_fig4_2_gains(results, 'outputs/fig4_2_gains.png')
plot_fig5_winter(results, winter_mask, weather, 'outputs/fig5_winter.png')
plot_fig6_shoulder(results, shoulder_mask, weather, 'outputs/fig6_shoulder.png')



Heat Flow Breakdown (S1)
--------------------------------------------------
Losses:
  Walls: 2829 kWh (23.3%)
  Roof: 1271 kWh (10.5%)
  Windows: 3793 kWh (31.3%)
  Infiltration: 2116 kWh (17.4%)
  Ventilation: 2116 kWh (17.4%)
  Total: 12124 kWh
Gains:
  Solar: 6497 kWh (39.3%)
  Internal: 1752 kWh (10.6%)
  Heating: 8275 kWh (50.1%)
  Total: 16524 kWh


# Task 3

In [7]:
# Time masks for weekly analysis
winter_mask = (weather['timestamp'] >= '2021-01-08') & (weather['timestamp'] < '2021-01-15')
shoulder_mask = (weather['timestamp'] >= '2021-10-07') & (weather['timestamp'] < '2021-10-14')

# Run sensitivity analysis
sens_df: pd.DataFrame = run_sensitivity(weather, winter_mask, shoulder_mask)
sens_table: pd.DataFrame = calc_sensitivity_table(sens_df)

print("\nSensitivity Analysis Results")
print("-" * 80)

for i, row_index in enumerate(range(len(sens_table))):
    row = sens_table.iloc[row_index]
    print(f"{i + 1} {row['Parameter']}: ")
    print (f"Winter {row['Q_w_min']:.1f} - {row['Q_w_max']:.1f} kWh (NSC = {row['NSC_w']:+.3f}), ")
    print (f"Shoulder {row['Q_s_min']:.1f} - {row['Q_s_max']:.1f} kWh (NSC = {row['NSC_s']:+.3f}) \n")

plot_fig7_scatter(sens_df, 'outputs/fig7_sensitivity.png')
plot_fig8_ranking(sens_table, 'outputs/fig8_ranking.png')


Sensitivity Analysis Results
--------------------------------------------------------------------------------
1 Orientation: 
Winter 265.9 - 299.7 kWh (NSC = +0.064), 
Shoulder 165.5 - 191.0 kWh (NSC = +0.077) 

2 Internal gains: 
Winter 236.0 - 281.1 kWh (NSC = -0.113), 
Shoulder 141.9 - 177.6 kWh (NSC = -0.144) 

3 Insulation k: 
Winter 240.2 - 290.1 kWh (NSC = +0.375), 
Shoulder 149.1 - 181.0 kWh (NSC = +0.386) 

4 Temp offset: 
Winter 229.6 - 303.0 kWh (NSC = -0.069), 
Shoulder 137.0 - 195.4 kWh (NSC = -0.088) 

5 Absorptance: 
Winter 265.3 - 266.4 kWh (NSC = -0.004), 
Shoulder 164.9 - 166.2 kWh (NSC = -0.007) 

6 Infiltration: 
Winter 234.9 - 318.7 kWh (NSC = +0.197), 
Shoulder 145.5 - 199.7 kWh (NSC = +0.205) 



# Task 4

In [8]:
from config import (
    BUILDING_VOLUME,
    AREA_ROOF, AREA_WALL_N, AREA_WALL_S, AREA_WALL_E, AREA_WALL_W, AREA_WINDOW,
    WINDOW_R, ALPHA, SHGC, INFILTRATION_ACH, INTERNAL_GAIN_W
)
from config_heavyweight import (
    WALL_R1_LW, WALL_R2_LW, WALL_R3_LW, WALL_CA_LW,
    WALL_R1_HW, WALL_R2_HW, WALL_R3_HW, WALL_CA_HW,
    ROOF_R1, ROOF_R2, ROOF_R3, ROOF_CA_LW, ROOF_CA_HW,
    C_WALL_LW, C_WALL_HW, C_ROOF_LW, C_ROOF_HW
)
from src.rc_model import run_rc_hourly
from src.rc_plots import (
    plot_glasgow_weather, 
    plot_rc_heatup_week, 
    plot_rc_shoulder_week,
    plot_rc_shoulder_heating, 
    plot_heating_histogram,
    plot_heating_comparison_histogram, 
    plot_monthly_peak_demand)

In [ ]:

def build_envelope_3r(wall_R1, wall_R2, wall_R3, wall_CA,
                      roof_R1, roof_R2, roof_R3, roof_CA) -> dict:
    """Build envelope parameters for 3-resistance RC model.

    R1: thermal mass to indoor (m²K/W)
    R2: thermal mass to external surface node (m²K/W)
    R3: external surface resistance (m²K/W)
    CA: thermal capacitance per unit area (kJ/m²K)
    """
    # Convert C/A from kJ/m²K to J/m²K
    wall_C = wall_CA * 1000
    roof_C = roof_CA * 1000

    # Roof
    roof = {
        'R1': roof_R1,
        'R2': roof_R2,
        'R3': roof_R3,
        'C': roof_C,
        'alpha': ALPHA,
        'area': AREA_ROOF,
    }

    # Walls - same R values per unit area, different areas
    wall_N = {'R1': wall_R1, 'R2': wall_R2, 'R3': wall_R3, 'C': wall_C, 'alpha': ALPHA, 'area': AREA_WALL_N}
    wall_S = {'R1': wall_R1, 'R2': wall_R2, 'R3': wall_R3, 'C': wall_C, 'alpha': ALPHA, 'area': AREA_WALL_S}
    wall_E = {'R1': wall_R1, 'R2': wall_R2, 'R3': wall_R3, 'C': wall_C, 'alpha': ALPHA, 'area': AREA_WALL_E}
    wall_W = {'R1': wall_R1, 'R2': wall_R2, 'R3': wall_R3, 'C': wall_C, 'alpha': ALPHA, 'area': AREA_WALL_W}

    # Windows (steady state, no thermal mass)
    windows = {
        'R': WINDOW_R,
        'area': AREA_WINDOW,
        'g': SHGC,
    }

    envelope = {
        'roof': roof,
        'walls': {'N': wall_N, 'S': wall_S, 'E': wall_E, 'W': wall_W},
        'windows': windows,
    }
    return envelope


def run_simulations() -> tuple:
    """Run RC simulations for lightweight and heavyweight buildings."""

    # Load weather
    print("Loading weather...")
    weather: pd.DataFrame = load_epw_weather(
        'weather_files/GBR_SCT_Glasgow.AP.031400_TMYx/GBR_SCT_Glasgow.AP.031400_TMYx.epw',
        'weather_files/solarposition_data_Glasgow.csv'
    )
    weather['timestamp'] = pd.to_datetime({
        'year': 2021,
        'month': weather['timestamp'].dt.month,
        'day': weather['timestamp'].dt.day,
        'hour': weather['timestamp'].dt.hour
    })
    weather = weather.sort_values('timestamp').reset_index(drop=True)

    # Building parameters (same as Task 1)
    air = AirSide(BUILDING_VOLUME, 0.0, 0.0, INFILTRATION_ACH)
    gains = InternalGains(INTERNAL_GAIN_W / 1000)
    T_set = 21.0
    vent_ACH = 0.5

    # Build envelopes with 3-resistance model
    env_lw: dict = build_envelope_3r(
        WALL_R1_LW, WALL_R2_LW, WALL_R3_LW, WALL_CA_LW,
        ROOF_R1, ROOF_R2, ROOF_R3, ROOF_CA_LW
    )
    env_hw: dict = build_envelope_3r(
        WALL_R1_HW, WALL_R2_HW, WALL_R3_HW, WALL_CA_HW,
        ROOF_R1, ROOF_R2, ROOF_R3, ROOF_CA_HW
    )

    print(f"Lightweight: wall C/A={WALL_CA_LW} kJ/m²K, roof C/A={ROOF_CA_LW} kJ/m²K")
    print(f"Heavyweight: wall C/A={WALL_CA_HW} kJ/m²K, roof C/A={ROOF_CA_HW} kJ/m²K")
    print(f"Total C: LW={C_WALL_LW/1e6:.2f}+{C_ROOF_LW/1e6:.2f} MJ/K, HW={C_WALL_HW/1e6:.2f}+{C_ROOF_HW/1e6:.2f} MJ/K")

    print("\nRunning simulations...")
    res_lw: pd.DataFrame = run_rc_hourly(weather, env_lw, air, T_set, vent_ACH, gains)
    res_hw: pd.DataFrame = run_rc_hourly(weather, env_hw, air, T_set, vent_ACH, gains)

    return res_lw, res_hw, weather


def print_results(res_lw: pd.DataFrame, res_hw: pd.DataFrame) -> None:
    """Print heating load and thermal mass statistics."""

    # Heating load stats (kW)
    Q_lw = res_lw['Q_heat_W'].values / 1000
    Q_hw = res_hw['Q_heat_W'].values / 1000

    # Total annual heating (kWh) - sum of Q * dt (15 min = 0.25 hr)
    dt_hours = 0.25
    total_lw = (Q_lw * dt_hours).sum()
    total_hw = (Q_hw * dt_hours).sum()

    print("\nHeating load (kW):")
    print(f"  Lightweight: mean={Q_lw.mean():.3f}, std={Q_lw.std():.3f}")
    print(f"  Heavyweight: mean={Q_hw.mean():.3f}, std={Q_hw.std():.3f}")
    print(f"\nAnnual heating (kWh):")
    print(f"  Lightweight: {total_lw:.1f} kWh")
    print(f"  Heavyweight: {total_hw:.1f} kWh ({(total_hw - total_lw) / total_lw * 100:+.1f}%)")

    print("\nThermal mass temps (mean/std):")
    surfaces = [
        ('Roof', 'T_C_roof'),
        ('North', 'T_C_N'),
        ('South', 'T_C_S'),
        ('East', 'T_C_E'),
        ('West', 'T_C_W'),
    ]
    for name, col in surfaces:
        lw_mean = res_lw[col].mean()
        lw_std = res_lw[col].std()
        hw_mean = res_hw[col].mean()
        hw_std = res_hw[col].std()
        print(f"  {name}: LW {lw_mean:.1f}C (std {lw_std:.2f}), HW {hw_mean:.1f}C (std {hw_std:.2f})")

In [10]:
# Building parameters (same as Task 1)
air = AirSide(BUILDING_VOLUME, 0.0, 0.0, INFILTRATION_ACH)
gains = InternalGains(INTERNAL_GAIN_W / 1000)
T_set = 21.0
vent_ACH = 0.5

# Build envelopes with 3-resistance model
env_lw: dict = build_envelope_3r(
    WALL_R1_LW, WALL_R2_LW, WALL_R3_LW, WALL_CA_LW,
    ROOF_R1, ROOF_R2, ROOF_R3, ROOF_CA_LW
)
env_hw: dict = build_envelope_3r(
    WALL_R1_HW, WALL_R2_HW, WALL_R3_HW, WALL_CA_HW,
    ROOF_R1, ROOF_R2, ROOF_R3, ROOF_CA_HW
)

print(f"Lightweight: wall C/A={WALL_CA_LW} kJ/m²K, roof C/A={ROOF_CA_LW} kJ/m²K")
print(f"Heavyweight: wall C/A={WALL_CA_HW} kJ/m²K, roof C/A={ROOF_CA_HW} kJ/m²K")

print("\nRunning simulations...")
res_lw: pd.DataFrame = run_rc_hourly(weather, env_lw, air, T_set, vent_ACH, gains)
res_hw: pd.DataFrame = run_rc_hourly(weather, env_hw, air, T_set, vent_ACH, gains)

Lightweight: wall C/A=9.576 kJ/m²K, roof C/A=7.98 kJ/m²K
Heavyweight: wall C/A=140.0 kJ/m²K, roof C/A=7.98 kJ/m²K

Running simulations...


In [11]:
# Heating load stats (kW)
Q_lw = res_lw['Q_heat_W'].values / 1000
Q_hw = res_hw['Q_heat_W'].values / 1000

# Total annual heating (kWh) - sum of Q * dt (15 min = 0.25 hr)
dt_hours = 0.25
total_lw = (Q_lw * dt_hours).sum()
total_hw = (Q_hw * dt_hours).sum()

print("\nHeating load (kW):")
print(f"  Lightweight: mean={Q_lw.mean():.3f}, std={Q_lw.std():.3f}")
print(f"  Heavyweight: mean={Q_hw.mean():.3f}, std={Q_hw.std():.3f}")

print(f"\nAnnual heating (kWh):")
print(f"  Lightweight: {total_lw:.1f} kWh")
print(f"  Heavyweight: {total_hw:.1f} kWh ({(total_hw - total_lw) / total_lw * 100:+.1f}%)")

print("\nThermal mass temps (mean/std):")
surfaces = [
    ('Roof', 'T_C_roof'),
    ('North', 'T_C_N'),
    ('South', 'T_C_S'),
    ('East', 'T_C_E'),
    ('West', 'T_C_W'),
]
for name, col in surfaces:
    lw_mean = res_lw[col].mean()
    lw_std = res_lw[col].std()
    hw_mean = res_hw[col].mean()
    hw_std = res_hw[col].std()
    print(f"  {name}: LW {lw_mean:.1f}C (std {lw_std:.2f}), HW {hw_mean:.1f}C (std {hw_std:.2f})")


Heating load (kW):
  Lightweight: mean=0.940, std=0.800
  Heavyweight: mean=0.913, std=0.777

Annual heating (kWh):
  Lightweight: 8236.7 kWh
  Heavyweight: 7999.6 kWh (-2.9%)

Thermal mass temps (mean/std):
  Roof: LW 20.4C (std 0.44), HW 20.4C (std 0.44)
  North: LW 19.8C (std 0.58), HW 19.1C (std 0.85)
  South: LW 19.9C (std 0.67), HW 19.3C (std 0.86)
  East: LW 19.9C (std 0.63), HW 19.2C (std 0.89)
  West: LW 19.9C (std 0.65), HW 19.2C (std 0.89)


In [12]:
plot_glasgow_weather(weather)
plot_rc_heatup_week(res_lw, res_hw, weather)
plot_rc_shoulder_week(res_lw, res_hw, weather)
plot_rc_shoulder_heating(res_lw, res_hw, weather)
plot_heating_histogram(res_lw, res_hw)
plot_heating_comparison_histogram(res_lw, res_hw)
plot_monthly_peak_demand(res_lw, res_hw)

Saved: outputs/glasgow_weather.png
Saved: outputs/rc_heatup_week.png
Saved: outputs/rc_shoulder_week.png
Saved: outputs/rc_shoulder_heating.png
Saved: outputs/rc_heating_histogram.png
Saved: outputs/rc_heating_comparison.png
Saved: outputs/rc_peak_demand.png
